In [1]:
import numpy as np
import pandas as pd
import calendar
from datetime import datetime as dt
import math


In [2]:
df = pd.read_csv('export.csv',sep=',',encoding ='latin1', parse_dates=['Completed on', 'Due date', 'Start date'])

In [3]:
df =df[['Id','Assigned to', 'Completed by','Completed on', 'Due date',  'Name', 'Start date', 'Tags', 'Workspaces']]
for col in df.select_dtypes(include=['datetime']):
    df[col] = df[col].dt.strftime('%d-%m-%Y')

In [4]:
df['CompletedWeek'] = pd.DatetimeIndex(df['Completed on']).week
df = df.fillna('NONE')

In [5]:
for col in df.select_dtypes(include=['object']):
    df[col] = df[col].apply(lambda x : "NONE" if x=="NaT" else x)

In [18]:
df.dtypes

Id                 int64
Assigned to       object
Completed by      object
Completed on      object
Due date          object
Name              object
Start date        object
Tags              object
Workspaces        object
CompletedWeek     object
status            object
assignedStatus    object
dtype: object

In [7]:
import calendar
import datetime
def getStatus(x):
    """ status information """
    status = ""
    if x['Completed on'] == "NONE":
        if x['Due date'] != "NONE":
            d3 = pd.to_datetime(x['Due date']) - datetime.datetime.now()
            if d3.days < 0:
                status = "OVERDUE"
            else:
                status = "OPEN"
        elif x['Due date'] == "NONE":
            status = "OPEN"
    elif x['Completed on'] != "NONE":
        if x['Due date'] != "NONE":
            d3 = pd.to_datetime(x['Completed on']) - pd.to_datetime(x['Due date'])
            if d3.days > 0:
                status = "OVERDUE_CLOSED"
            else:
                status = "CLOSED"
        elif x['Due date'] == "NONE":
            status = "CLOSED"
    return status


In [8]:
df['status'] = df.apply(getStatus, axis=1)

In [11]:
df.head()

Id   Assigned to     Completed by Completed on    Due date  \
0   80  Shini Sharma             NONE         NONE  21-04-2017   
1   82          NONE             NONE         NONE  28-04-2017   
2  114          NONE             NONE         NONE        NONE   
3  106          NONE             NONE         NONE  04-06-2017   
4   64  Shini Sharma  Joerg Engelmann   04-07-2017  03-10-2017   

                                           Name  Start date    Tags  \
0  load SO information into MAXIMO/update Table  03-01-2017      IT   
1  load SO information into MAXIMO/update Table  17-04-2017      IT   
2                                      Resource  03-01-2017    NONE   
3                       session with TCS and RR  04-06-2017    NONE   
4                  download & process DEIS data  03-01-2017  MAXIMO   

                       Workspaces CompletedWeek          status assignedStatus  
0      projects/SAPSIMO/Data Load          NONE         OVERDUE       ASSIGNED  
1      projects/SAPSIMO/Data Load          NONE         OVERDUE   NOT ASSIGNED  
2                weekly follow up          NONE            OPEN   NOT ASSIGNED  
3                weekly follow up          NONE         OVERDUE   NOT ASSIGNED  
4  operational process/#3 monthly            14  OVERDUE_CLOSED       ASSIGNED

In [10]:
df['assignedStatus'] = df.apply(lambda x: "NOT ASSIGNED" if x['Assigned to'] == "NONE" else "ASSIGNED", axis=1)

In [15]:
df1 =pd.DataFrame({'count' : df.groupby( ["CompletedWeek"] ).size()}).reset_index()


In [17]:
df1.head()

CompletedWeek  count
0             1     88
1             2     73
2             3     65
3             4    505
4             5    110

In [25]:
def setMonthYear(row):
    """ month-year"""
    dateArr = []
    if row['Start date'] != "NONE":
        d3 = pd.to_datetime(row['Start date'])
        dateArr.append(str(calendar.month_abbr[d3.month]))
        dateArr.append(str(d3.year))
    return "-".join(dateArr)

In [38]:
df.dtypes

Id                 int64
Assigned to       object
Completed by      object
Completed on      object
Due date          object
Name              object
Start date        object
Tags              object
Workspaces        object
CompletedWeek     object
status            object
assignedStatus    object
month_year        object
dtype: object

In [26]:
df['month_year'] = df.apply(setMonthYear, axis=1)

In [27]:
df.head()

Id   Assigned to     Completed by Completed on    Due date  \
0   80  Shini Sharma             NONE         NONE  21-04-2017   
1   82          NONE             NONE         NONE  28-04-2017   
2  114          NONE             NONE         NONE        NONE   
3  106          NONE             NONE         NONE  04-06-2017   
4   64  Shini Sharma  Joerg Engelmann   04-07-2017  03-10-2017   

                                           Name  Start date    Tags  \
0  load SO information into MAXIMO/update Table  03-01-2017      IT   
1  load SO information into MAXIMO/update Table  17-04-2017      IT   
2                                      Resource  03-01-2017    NONE   
3                       session with TCS and RR  04-06-2017    NONE   
4                  download & process DEIS data  03-01-2017  MAXIMO   

                       Workspaces CompletedWeek          status  \
0      projects/SAPSIMO/Data Load          NONE         OVERDUE   
1      projects/SAPSIMO/Data Load          NONE         OVERDUE   
2                weekly follow up          NONE            OPEN   
3                weekly follow up          NONE         OVERDUE   
4  operational process/#3 monthly            14  OVERDUE_CLOSED   

  assignedStatus month_year  
0       ASSIGNED   Mar-2017  
1   NOT ASSIGNED   Apr-2017  
2   NOT ASSIGNED   Mar-2017  
3   NOT ASSIGNED   Apr-2017  
4       ASSIGNED   Mar-2017

In [34]:
def setMonth(row):
    """month"""
    if row['Start date'] != "NONE":
        d4 = pd.to_datetime(row['Start date'])
    return d4.month



UnboundLocalError: ("local variable 'd4' referenced before assignment", 'occurred at index 12')

In [36]:
df['month'] = df.apply(setMonth, axis=1)

UnboundLocalError: ("local variable 'd4' referenced before assignment", 'occurred at index 12')

In [41]:
def setMonth(x):
    """month"""
    if x['Start date'] != "NONE":
        d1 = pd.to_datetime(x['Start date'])
        return d1.month

In [42]:
df['month'] = df.apply(setMonth,axis=1)

In [43]:

def setYear(row):
    """year"""
    if row['Start date'] != "NONE":
        d5 = pd.to_datetime(row['Start date'])
        return d5.year


def setWorkspace(row):
    """workspace data"""
    workspace = ""
    if isinstance(row['Workspaces'], str):
        if 'operational' in row['Workspaces']:
            workspace = "Operational"
        elif 'projects' in row['Workspaces']:
            workspace = "Project"
        else:
            workspace = "None"
    else:
        workspace = "None"
    return workspace

In [44]:
df['year'] = df.apply(setYear, axis=1)
df['workspaceGroup'] = df.apply(setWorkspace, axis=1)
df = df.sort_values(['month','year'])

In [45]:
df.head()

Id    Assigned to             Completed by Completed on    Due date  \
2302   254   Shini Sharma          Joerg Engelmann   17-07-2018        NONE   
957   4401    Kiran Kumar  Sakthi Mainthan Abishek   15-12-2017  01-05-2018   
960   4826  Ipsita Bhatta            Ipsita Bhatta   01-05-2018  01-05-2018   
961   4854  Ipsita Bhatta            Ipsita Bhatta   01-05-2018  01-05-2018   
962   4881  Ipsita Bhatta            Ipsita Bhatta   01-05-2018  01-05-2018   

                                                   Name  Start date  \
2302                                MAXIMO operator fix  01-01-2017   
957   Provide completed Rotable sheet for processing...  01-01-2018   
960              SB Embodiment and SB Campaign activity  01-01-2018   
961                              Delta Event Management  01-01-2018   
962          New aircraft Creation and Missing Operator  01-01-2018   

             Tags                      Workspaces CompletedWeek  status  \
2302       MAXIMO              projects/dev tasks            29  CLOSED   
957   MAXIMO - SE  operational process/#3 monthly            50  CLOSED   
960            SE   operational process/#2 weekly             1  CLOSED   
961            SE   operational process/#2 weekly             1  CLOSED   
962            SE   operational process/#2 weekly             1  CLOSED   

     assignedStatus month_year  month    year workspaceGroup  
2302       ASSIGNED   Jan-2017    1.0  2017.0        Project  
957        ASSIGNED   Jan-2018    1.0  2018.0    Operational  
960        ASSIGNED   Jan-2018    1.0  2018.0    Operational  
961        ASSIGNED   Jan-2018    1.0  2018.0    Operational  
962        ASSIGNED   Jan-2018    1.0  2018.0    Operational

In [50]:
groupByMonthData =pd.DataFrame({'count' : df.groupby( ['month_year','workspaceGroup'] ).size()}).reset_index()

In [51]:
groupByMonthData = groupByMonthData.pivot(index='month_year',columns='workspaceGroup',values='count')

In [52]:
groupByMonthData = groupByMonthData.fillna(0)
groupByMonthData.index = pd.to_datetime(groupByMonthData.index)
groupByMonthData.sort_index(inplace=True)
groupByMonthData.index = groupByMonthData.index.strftime('%b-%y')

In [53]:
groupByMonthData

workspaceGroup  None  Operational  Project
Jan-17           0.0          0.0      1.0
Feb-17           2.0          5.0      9.0
Mar-17           3.0         23.0     14.0
Apr-17          13.0         56.0     20.0
May-17           1.0         28.0     17.0
Jun-17           5.0         49.0     79.0
Jul-17           8.0        108.0     68.0
Aug-17          16.0         91.0     46.0
Sep-17           9.0        105.0     39.0
Oct-17           5.0        112.0     59.0
Nov-17           6.0         99.0     23.0
Dec-17           1.0         85.0     62.0
Jan-18           2.0         94.0     49.0
Feb-18           4.0         83.0     50.0
Mar-18           1.0         92.0     21.0
Apr-18           2.0        100.0     14.0
May-18           1.0        102.0     42.0
Jun-18           1.0        106.0     50.0
Jul-18           1.0        117.0     20.0
Aug-18           1.0        107.0     37.0
Sep-18           1.0        109.0     59.0
Oct-18           1.0        124.0     34.0
Nov-18           1.0        106.0     16.0
Dec-18           3.0        123.0     22.0
Jan-19           4.0        103.0     34.0
Feb-19           1.0         86.0      7.0
Mar-19           4.0         83.0     24.0
Apr-19          10.0         83.0     32.0
May-19          10.0         82.0     22.0
Jun-19          16.0        115.0     11.0
Jul-19          11.0        135.0     22.0
Aug-19           9.0        128.0     24.0
Sep-19           8.0        150.0     12.0
Oct-19           7.0        154.0     25.0
Nov-19           8.0        172.0     36.0
Dec-19          13.0        162.0     21.0
Jan-20           5.0        165.0      7.0
Feb-20           3.0         59.0      2.0
Mar-20           1.0         28.0      0.0
Apr-20           2.0         16.0      0.0
May-20           1.0          1.0      0.0
Jun-20           0.0          1.0      0.0
Jul-20           0.0          1.0      0.0
Aug-20           0.0          1.0      0.0
Apr-49           0.0          3.0      0.0
May-49           0.0          4.0      0.0
Jun-49           0.0          4.0      0.0
Jul-49           0.0          5.0      0.0
Aug-49           0.0          4.0      0.0
Sep-49           0.0          5.0      0.0
Oct-49           0.0          4.0      0.0
Nov-49           0.0          4.0      0.0
NaT             94.0         30.0    162.0

In [54]:
df

Id              Assigned to             Completed by Completed on  \
2302    254             Shini Sharma          Joerg Engelmann   17-07-2018   
957    4401              Kiran Kumar  Sakthi Mainthan Abishek   15-12-2017   
960    4826            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
961    4854            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
962    4881            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
...     ...                      ...                      ...          ...   
4656  74474               Siva Koppu                     NONE         NONE   
4657  74475               Siva Koppu                     NONE         NONE   
5123  74833  Sakthi Mainthan Abishek    Saran Sadasivamoorthi   23-01-2020   
5307  70741                 Anjana G                 Anjana G   27-01-2020   
5350  77133                 Anjana G                     NONE         NONE   

        Due date                                               Name  \
2302        NONE                                MAXIMO operator fix   
957   01-05-2018  Provide completed Rotable sheet for processing...   
960   01-05-2018             SB Embodiment and SB Campaign activity   
961   01-05-2018                             Delta Event Management   
962   01-05-2018         New aircraft Creation and Missing Operator   
...          ...                                                ...   
4656        NONE                                 Maximo EDRR Report   
4657        NONE                     PZ Change Request - DI Monitor   
5123        NONE                  Consolidate all CMP weekly sheets   
5307        NONE          EN life conversion script for Gazoomobile   
5350        NONE                           Campaign report to PCB50   

      Start date           Tags                          Workspaces  \
2302  01-01-2017         MAXIMO                  projects/dev tasks   
957   01-01-2018    MAXIMO - SE      operational process/#3 monthly   
960   01-01-2018             SE       operational process/#2 weekly   
961   01-01-2018             SE       operational process/#2 weekly   
962   01-01-2018             SE       operational process/#2 weekly   
...          ...            ...                                 ...   
4656        NONE  #CLE - COGNOS                  projects/dev tasks   
4657        NONE  #CLE - COGNOS                  projects/dev tasks   
5123        NONE           NONE                                NONE   
5307        NONE           NONE  projects/DataSupplyChainAutomation   
5350        NONE             IT    projects/39_DeltaDataSupplyChain   

     CompletedWeek  status assignedStatus month_year  month    year  \
2302            29  CLOSED       ASSIGNED   Jan-2017    1.0  2017.0   
957             50  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
960              1  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
961              1  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
962              1  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
...            ...     ...            ...        ...    ...     ...   
4656          NONE    OPEN       ASSIGNED               NaN     NaN   
4657          NONE    OPEN       ASSIGNED               NaN     NaN   
5123             4  CLOSED       ASSIGNED               NaN     NaN   
5307             5  CLOSED       ASSIGNED               NaN     NaN   
5350          NONE    OPEN       ASSIGNED               NaN     NaN   

     workspaceGroup  
2302        Project  
957     Operational  
960     Operational  
961     Operational  
962     Operational  
...             ...  
4656        Project  
4657        Project  
5123           None  
5307        Project  
5350        Project  

[5399 rows x 16 columns]

In [55]:
withStartDate = df[df['Start date'] != 'NONE']
projectedTasks = withStartDate[pd.to_datetime(withStartDate['Start date'], errors='ignore') > dt.now()]

In [57]:
projectedTasks[~projectedTasks['Tags'].str.contains('status')]

Id              Assigned to           Completed by Completed on  \
4557  72845             Shini Sharma                   NONE         NONE   
4558  72846             Shini Sharma                   NONE         NONE   
4577  72848  Sakthi Mainthan Abishek                   NONE         NONE   
4578  72856  Sakthi Mainthan Abishek                   NONE         NONE   
4876  74771  Sakthi Mainthan Abishek  Saran Sadasivamoorthi   23-01-2020   
...     ...                      ...                    ...          ...   
4301  68285              Sudhakar CD            Sudhakar CD   12-06-2019   
4582  69562              Sudhakar CD            Sudhakar CD   01-03-2020   
4602  72861              Sudhakar CD            Sudhakar CD   01-10-2020   
5320  76066              Sudhakar CD                   NONE         NONE   
5321  73706              Sudhakar CD            Sudhakar CD   30-01-2020   

        Due date                                               Name  \
4557  27-02-2020                        Download CAMP data from FTP   
4558  27-02-2020                        Download CAMP data from FTP   
4577  27-02-2020  Provide cleansed CAMP input data dump for proc...   
4578  27-02-2020  Provide cleansed CAMP input data dump for proc...   
4876        NONE  Provide cleansed CAMP input data dump for proc...   
...          ...                                                ...   
4301        NONE                                     AST Event load   
4582        NONE                                     AST Event load   
4602        NONE                                     AST Event load   
5320        NONE                                     AST Event load   
5321        NONE                                     AST Event load   

      Start date    Tags                      Workspaces CompletedWeek  \
4557  24-02-2020  MAXIMO  operational process/#3 monthly          NONE   
4558  24-02-2020  MAXIMO  operational process/#3 monthly          NONE   
4577  24-02-2020  MAXIMO  operational process/#3 monthly          NONE   
4578  24-02-2020  MAXIMO  operational process/#3 monthly          NONE   
4876  24-02-2020  MAXIMO  operational process/#3 monthly             4   
...          ...     ...                             ...           ...   
4301  28-10-2049  MAXIMO   operational process/#2 weekly            49   
4582  11-04-2049  MAXIMO   operational process/#2 weekly             1   
4602  11-11-2049  MAXIMO   operational process/#2 weekly             2   
5320  25-11-2049  MAXIMO   operational process/#2 weekly          NONE   
5321  18-11-2049  MAXIMO   operational process/#2 weekly             5   

      status assignedStatus month_year  month    year workspaceGroup  
4557    OPEN       ASSIGNED   Feb-2020    2.0  2020.0    Operational  
4558    OPEN       ASSIGNED   Feb-2020    2.0  2020.0    Operational  
4577    OPEN       ASSIGNED   Feb-2020    2.0  2020.0    Operational  
4578    OPEN       ASSIGNED   Feb-2020    2.0  2020.0    Operational  
4876  CLOSED       ASSIGNED   Feb-2020    2.0  2020.0    Operational  
...      ...            ...        ...    ...     ...            ...  
4301  CLOSED       ASSIGNED   Oct-2049   10.0  2049.0    Operational  
4582  CLOSED       ASSIGNED   Nov-2049   11.0  2049.0    Operational  
4602  CLOSED       ASSIGNED   Nov-2049   11.0  2049.0    Operational  
5320    OPEN       ASSIGNED   Nov-2049   11.0  2049.0    Operational  
5321  CLOSED       ASSIGNED   Nov-2049   11.0  2049.0    Operational  

[104 rows x 16 columns]

In [58]:
closeTasks = withStartDate[withStartDate['status'].str.contains('CLOSED')]
closeTasks = closeTasks[pd.to_datetime(closeTasks['Start date'], errors='ignore') < dt.now()]
closeTasks

Id              Assigned to             Completed by Completed on  \
2302    254             Shini Sharma          Joerg Engelmann   17-07-2018   
957    4401              Kiran Kumar  Sakthi Mainthan Abishek   15-12-2017   
960    4826            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
961    4854            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
962    4881            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
...     ...                      ...                      ...          ...   
5263  75164  Sakthi Mainthan Abishek              Sudhakar CD   24-01-2020   
5272  75155  Sakthi Mainthan Abishek              Sudhakar CD   24-01-2020   
5286  74933              Kiran Kumar            Siva Santhosh   24-01-2020   
5319  70845         Madhan Sundarraj         Madhan Sundarraj   28-01-2020   
5359  73826            Ipsita Bhatta    Saran Sadasivamoorthi   02-06-2020   

        Due date                                               Name  \
2302        NONE                                MAXIMO operator fix   
957   01-05-2018  Provide completed Rotable sheet for processing...   
960   01-05-2018             SB Embodiment and SB Campaign activity   
961   01-05-2018                             Delta Event Management   
962   01-05-2018         New aircraft Creation and Missing Operator   
...          ...                                                ...   
5263  23-12-2019                     Download Rotable monthly sheet   
5272  25-12-2019  Provide cleansed CAMP input data dump for proc...   
5286  18-12-2019  Provide completed Rotable sheet for processing...   
5319  01-02-2020             Creation of event from ASC into Maximo   
5359  13-01-2020                           AMC Daily Shipments File   

      Start date         Tags                      Workspaces CompletedWeek  \
2302  01-01-2017       MAXIMO              projects/dev tasks            29   
957   01-01-2018  MAXIMO - SE  operational process/#3 monthly            50   
960   01-01-2018           SE   operational process/#2 weekly             1   
961   01-01-2018           SE   operational process/#2 weekly             1   
962   01-01-2018           SE   operational process/#2 weekly             1   
...          ...          ...                             ...           ...   
5263  18-12-2019       MAXIMO  operational process/#3 monthly             4   
5272  23-12-2019       MAXIMO  operational process/#3 monthly             4   
5286  12-11-2019       MAXIMO  operational process/#3 monthly             4   
5319  12-02-2019           SE             operational process             5   
5359  30-12-2019           SE  operational process/#3 monthly             6   

              status assignedStatus month_year  month    year workspaceGroup  
2302          CLOSED       ASSIGNED   Jan-2017    1.0  2017.0        Project  
957           CLOSED       ASSIGNED   Jan-2018    1.0  2018.0    Operational  
960           CLOSED       ASSIGNED   Jan-2018    1.0  2018.0    Operational  
961           CLOSED       ASSIGNED   Jan-2018    1.0  2018.0    Operational  
962           CLOSED       ASSIGNED   Jan-2018    1.0  2018.0    Operational  
...              ...            ...        ...    ...     ...            ...  
5263  OVERDUE_CLOSED       ASSIGNED   Dec-2019   12.0  2019.0    Operational  
5272  OVERDUE_CLOSED       ASSIGNED   Dec-2019   12.0  2019.0    Operational  
5286  OVERDUE_CLOSED       ASSIGNED   Dec-2019   12.0  2019.0    Operational  
5319  OVERDUE_CLOSED       ASSIGNED   Dec-2019   12.0  2019.0    Operational  
5359  OVERDUE_CLOSED       ASSIGNED   Dec-2019   12.0  2019.0    Operational  

[4435 rows x 16 columns]

In [60]:
noStartDate = df[df['Start date'] == 'NONE']
noStartDateCompleted = noStartDate[(noStartDate['status']=='CLOSED')]
noStartDateOpen = noStartDate[(noStartDate['status']=='OPEN')]
withStartDate = df[df['Start date'] != 'NONE']
projectedTasks = withStartDate[pd.to_datetime(withStartDate['Start date'], errors='ignore') > dt.now()]
projectedTasksWoBl = projectedTasks[~projectedTasks['Tags'].str.contains('status')]
overdueYetToComplete = withStartDate[withStartDate['status']=='OVERDUE_OPEN']
overdueYetToCompleteWoBl = overdueYetToComplete[~overdueYetToComplete['Tags'].str.contains('status')]
overdueComplete = withStartDate[withStartDate['status']=='OVERDUE_CLOSED']
overdueCompleteWoBl = overdueComplete[~overdueComplete['Tags'].str.contains('status')]
openTasks = withStartDate[withStartDate['status'].str.contains('OPEN')]
openTasks = openTasks[pd.to_datetime(openTasks['Start date'], errors='ignore') < dt.now()]
openTasksWoBl = openTasks[~openTasks['Tags'].str.contains('status')]
openTasksWoBl = openTasksWoBl[~openTasksWoBl['Tags'].str.contains('Bug')]
closeTasks = withStartDate[withStartDate['status'].str.contains('CLOSED')]
closeTasks = closeTasks[pd.to_datetime(closeTasks['Start date'], errors='ignore') < dt.now()]
closeTasksWoBl = closeTasks[~closeTasks['Tags'].str.contains('status')]
closeTasksWoBl = closeTasksWoBl[~closeTasksWoBl['Tags'].str.contains('Bug')]


completeOpenList = pd.concat([projectedTasksWoBl,overdueYetToCompleteWoBl,openTasksWoBl]).drop_duplicates().reset_index(drop=True)

In [61]:
completeOpenList

Id              Assigned to           Completed by Completed on  \
0    72845             Shini Sharma                   NONE         NONE   
1    72846             Shini Sharma                   NONE         NONE   
2    72848  Sakthi Mainthan Abishek                   NONE         NONE   
3    72856  Sakthi Mainthan Abishek                   NONE         NONE   
4    74771  Sakthi Mainthan Abishek  Saran Sadasivamoorthi   23-01-2020   
..     ...                      ...                    ...          ...   
325  69831           Vinay Kushwaha                   NONE         NONE   
326  69218           Vinay Kushwaha                   NONE         NONE   
327  69217             Hidyatulla S                   NONE         NONE   
328  68918              Debanu Saha                   NONE         NONE   
329  70850             Hidyatulla S                   NONE         NONE   

       Due date                                               Name  \
0    27-02-2020                        Download CAMP data from FTP   
1    27-02-2020                        Download CAMP data from FTP   
2    27-02-2020  Provide cleansed CAMP input data dump for proc...   
3    27-02-2020  Provide cleansed CAMP input data dump for proc...   
4          NONE  Provide cleansed CAMP input data dump for proc...   
..          ...                                                ...   
325        NONE                             BPD KPI setup for 2020   
326        NONE         add authentication and tracking of changes   
327        NONE  DRQ: concept for notification of urgent reques...   
328        NONE                               CCDC process upgrade   
329        NONE               EHM/WIP solve issues with connection   

     Start date                          Tags  \
0    24-02-2020                        MAXIMO   
1    24-02-2020                        MAXIMO   
2    24-02-2020                        MAXIMO   
3    24-02-2020                        MAXIMO   
4    24-02-2020                        MAXIMO   
..          ...                           ...   
325  12-12-2019                            IT   
326  12-05-2019                            IT   
327  12-05-2019                            IT   
328  12-03-2019  Data Engineering and Science   
329  19-12-2019  Data Engineering and Science   

                             Workspaces CompletedWeek  status assignedStatus  \
0        operational process/#3 monthly          NONE    OPEN       ASSIGNED   
1        operational process/#3 monthly          NONE    OPEN       ASSIGNED   
2        operational process/#3 monthly          NONE    OPEN       ASSIGNED   
3        operational process/#3 monthly          NONE    OPEN       ASSIGNED   
4        operational process/#3 monthly             4  CLOSED       ASSIGNED   
..                                  ...           ...     ...            ...   
325          projects/BPD KPI Dashboard          NONE    OPEN       ASSIGNED   
326          projects/BPD KPI Dashboard          NONE    OPEN       ASSIGNED   
327                  projects/dev tasks          NONE    OPEN       ASSIGNED   
328  projects/DataSupplyChainAutomation          NONE    OPEN       ASSIGNED   
329  projects/DataSupplyChainAutomation          NONE    OPEN       ASSIGNED   

    month_year  month    year workspaceGroup  
0     Feb-2020    2.0  2020.0    Operational  
1     Feb-2020    2.0  2020.0    Operational  
2     Feb-2020    2.0  2020.0    Operational  
3     Feb-2020    2.0  2020.0    Operational  
4     Feb-2020    2.0  2020.0    Operational  
..         ...    ...     ...            ...  
325   Dec-2019   12.0  2019.0        Project  
326   Dec-2019   12.0  2019.0        Project  
327   Dec-2019   12.0  2019.0        Project  
328   Dec-2019   12.0  2019.0        Project  
329   Dec-2019   12.0  2019.0        Project  

[330 rows x 16 columns]

In [62]:
def dfDateFilter(start_date,end_date, df):
    if start_date is not None and end_date is not None:
        df1 = df[(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') > start_date)&(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') < end_date)]
    elif start_date is not None and end_date is None:
        df1 = df[(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') > start_date)]
    elif start_date is None and end_date is not None:
        df1 = df[(pd.to_datetime(df['Start date'], dayfirst=True, errors='ignore') < end_date)]
    else:
        df1 = df
    return df1

In [63]:
def groupDf(df):
    groupedByTagsWs = df
    groupedByTagsWs['startDateWeek'] = pd.DatetimeIndex(groupedByTagsWs['Start date']).week
    groupedByTagsWs1 = pd.DataFrame({'count' : groupedByTagsWs.groupby(['Tags','startDateWeek']).size()}).reset_index()
    groupedByTagsWs1 = groupedByTagsWs1.pivot(index='startDateWeek',columns='Tags',values='count')
    return groupedByTagsWs1

In [66]:
dfDateFilter('01-01-2020','01-03-2020',df)

Id       Assigned to             Completed by Completed on  \
2441  27410       Kiran Kumar  Sakthi Mainthan Abishek   21-12-2018   
2579  27420       Kiran Kumar  Sakthi Mainthan Abishek   02-01-2019   
3019  28283       Debanu Saha              Debanu Saha   05-06-2019   
4390  70391              NONE    Saran Sadasivamoorthi   13-12-2019   
4583  72865      Kriti Sharma                     NONE         NONE   
5311  71653  Madhan Sundarraj         Madhan Sundarraj   28-01-2020   
5343  72808  Madhan Sundarraj         Madhan Sundarraj   02-03-2020   
5346  75997  Madhan Sundarraj         Madhan Sundarraj   02-03-2020   

        Due date                                               Name  \
2441  01-10-2019  Provide completed Rotable sheet for processing...   
2579  01-10-2019  Provide completed Rotable sheet for processing...   
3019        NONE                             load MSVL into SAPSIMO   
4390        NONE       Work Order Completion & SV report attachment   
4583  17-01-2020                  OSD sftp download data Processing   
5311  31-01-2020                               LRU SE - CDS TRACKER   
5343  02-03-2020             Creation of UTAS WIP event into Maximo   
5346  02-03-2020             Creation of event from ASC into Maximo   

      Start date                               Tags  \
2441  01-03-2019                             MAXIMO   
2579  01-03-2019                             MAXIMO   
3019  01-03-2019       Data Engineering and Science   
4390  01-03-2019                                 SE   
4583  01-02-2020  Data Engineering and Science - IT   
5311  01-02-2020                                 SE   
5343  01-02-2020                                 SE   
5346  01-02-2020                                 SE   

                              Workspaces CompletedWeek          status  \
2441      operational process/#3 monthly            51          CLOSED   
2579      operational process/#3 monthly             5  OVERDUE_CLOSED   
3019          projects/SAPSIMO/Data Load            19          CLOSED   
4390                 operational process            50          CLOSED   
4583  projects/DataSupplyChainAutomation          NONE         OVERDUE   
5311     operational process/#5 on going             5          CLOSED   
5343                 operational process             6          CLOSED   
5346                 operational process             6          CLOSED   

     assignedStatus month_year  month    year workspaceGroup  
2441       ASSIGNED   Jan-2019    1.0  2019.0    Operational  
2579       ASSIGNED   Jan-2019    1.0  2019.0    Operational  
3019       ASSIGNED   Jan-2019    1.0  2019.0        Project  
4390   NOT ASSIGNED   Jan-2019    1.0  2019.0    Operational  
4583       ASSIGNED   Jan-2020    1.0  2020.0        Project  
5311       ASSIGNED   Jan-2020    1.0  2020.0    Operational  
5343       ASSIGNED   Jan-2020    1.0  2020.0    Operational  
5346       ASSIGNED   Jan-2020    1.0  2020.0    Operational

In [67]:
projectedTasks.count()

Id                104
Assigned to       104
Completed by      104
Completed on      104
Due date          104
Name              104
Start date        104
Tags              104
Workspaces        104
CompletedWeek     104
status            104
assignedStatus    104
month_year        104
month             104
year              104
workspaceGroup    104
dtype: int64

In [68]:
df

Id              Assigned to             Completed by Completed on  \
2302    254             Shini Sharma          Joerg Engelmann   17-07-2018   
957    4401              Kiran Kumar  Sakthi Mainthan Abishek   15-12-2017   
960    4826            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
961    4854            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
962    4881            Ipsita Bhatta            Ipsita Bhatta   01-05-2018   
...     ...                      ...                      ...          ...   
4656  74474               Siva Koppu                     NONE         NONE   
4657  74475               Siva Koppu                     NONE         NONE   
5123  74833  Sakthi Mainthan Abishek    Saran Sadasivamoorthi   23-01-2020   
5307  70741                 Anjana G                 Anjana G   27-01-2020   
5350  77133                 Anjana G                     NONE         NONE   

        Due date                                               Name  \
2302        NONE                                MAXIMO operator fix   
957   01-05-2018  Provide completed Rotable sheet for processing...   
960   01-05-2018             SB Embodiment and SB Campaign activity   
961   01-05-2018                             Delta Event Management   
962   01-05-2018         New aircraft Creation and Missing Operator   
...          ...                                                ...   
4656        NONE                                 Maximo EDRR Report   
4657        NONE                     PZ Change Request - DI Monitor   
5123        NONE                  Consolidate all CMP weekly sheets   
5307        NONE          EN life conversion script for Gazoomobile   
5350        NONE                           Campaign report to PCB50   

      Start date           Tags                          Workspaces  \
2302  01-01-2017         MAXIMO                  projects/dev tasks   
957   01-01-2018    MAXIMO - SE      operational process/#3 monthly   
960   01-01-2018             SE       operational process/#2 weekly   
961   01-01-2018             SE       operational process/#2 weekly   
962   01-01-2018             SE       operational process/#2 weekly   
...          ...            ...                                 ...   
4656        NONE  #CLE - COGNOS                  projects/dev tasks   
4657        NONE  #CLE - COGNOS                  projects/dev tasks   
5123        NONE           NONE                                NONE   
5307        NONE           NONE  projects/DataSupplyChainAutomation   
5350        NONE             IT    projects/39_DeltaDataSupplyChain   

     CompletedWeek  status assignedStatus month_year  month    year  \
2302            29  CLOSED       ASSIGNED   Jan-2017    1.0  2017.0   
957             50  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
960              1  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
961              1  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
962              1  CLOSED       ASSIGNED   Jan-2018    1.0  2018.0   
...            ...     ...            ...        ...    ...     ...   
4656          NONE    OPEN       ASSIGNED               NaN     NaN   
4657          NONE    OPEN       ASSIGNED               NaN     NaN   
5123             4  CLOSED       ASSIGNED               NaN     NaN   
5307             5  CLOSED       ASSIGNED               NaN     NaN   
5350          NONE    OPEN       ASSIGNED               NaN     NaN   

     workspaceGroup  
2302        Project  
957     Operational  
960     Operational  
961     Operational  
962     Operational  
...             ...  
4656        Project  
4657        Project  
5123           None  
5307        Project  
5350        Project  

[5399 rows x 16 columns]